In [10]:
p = 97
x_data = list(range(p))
y_data = list(range(1, p))

In [24]:
45 % 97

45

In [16]:
def get_label(x_data, y_data, x_idx, y_idx, p, operation):
    x, y = x_data[x_idx], y_data[y_idx]
    return operation(x, y, p) 

def div_mod(x, y, p):
    res = (x/y) % p
    print(f"x: {x}, y: {y}, res: {res}")
    return res


In [29]:
def test_div_mod():
    expected_results = [
        (0, 5, 0),
        (5, 5, 1),
        (90, 3, 30),
        (90, 2, 45)
    ]
    for x, y, result in expected_results:
        assert div_mod(x, y, 97) == result

test_div_mod()

x: 0, y: 5, res: 0.0
x: 5, y: 5, res: 1.0
x: 90, y: 3, res: 30.0
x: 90, y: 2, res: 45.0


In [18]:
for i in range(len(x_data)):
    for j in range(len(y_data)):
        get_label(x_data, y_data, i, j, p, div_mod)

x: 0, y: 1, res: 0.0
x: 0, y: 2, res: 0.0
x: 0, y: 3, res: 0.0
x: 0, y: 4, res: 0.0
x: 0, y: 5, res: 0.0
x: 0, y: 6, res: 0.0
x: 0, y: 7, res: 0.0
x: 0, y: 8, res: 0.0
x: 0, y: 9, res: 0.0
x: 0, y: 10, res: 0.0
x: 0, y: 11, res: 0.0
x: 0, y: 12, res: 0.0
x: 0, y: 13, res: 0.0
x: 0, y: 14, res: 0.0
x: 0, y: 15, res: 0.0
x: 0, y: 16, res: 0.0
x: 0, y: 17, res: 0.0
x: 0, y: 18, res: 0.0
x: 0, y: 19, res: 0.0
x: 0, y: 20, res: 0.0
x: 0, y: 21, res: 0.0
x: 0, y: 22, res: 0.0
x: 0, y: 23, res: 0.0
x: 0, y: 24, res: 0.0
x: 0, y: 25, res: 0.0
x: 0, y: 26, res: 0.0
x: 0, y: 27, res: 0.0
x: 0, y: 28, res: 0.0
x: 0, y: 29, res: 0.0
x: 0, y: 30, res: 0.0
x: 0, y: 31, res: 0.0
x: 0, y: 32, res: 0.0
x: 0, y: 33, res: 0.0
x: 0, y: 34, res: 0.0
x: 0, y: 35, res: 0.0
x: 0, y: 36, res: 0.0
x: 0, y: 37, res: 0.0
x: 0, y: 38, res: 0.0
x: 0, y: 39, res: 0.0
x: 0, y: 40, res: 0.0
x: 0, y: 41, res: 0.0
x: 0, y: 42, res: 0.0
x: 0, y: 43, res: 0.0
x: 0, y: 44, res: 0.0
x: 0, y: 45, res: 0.0
x: 0, y: 46, res: 0